In [ ]:
import glob
import os
import sys
import numpy as np
import cv2
import random
import time


In [ ]:
try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass
import carla

In [ ]:
actor_list=[]
client=carla.Client('localhost',2000)
client=carla.Client('localhost',2000)
client.set_timeout(2000.0)

In [ ]:


world=client.get_world()
bpl=world.get_blueprint_library()
bp=bpl.filter('model3')[0]
print(bp)
spp=world.get_map().get_spawn_points()
sp=spp[10]
vl=world.spawn_actor(bp,sp)
actor_list.append(vl)

In [ ]:
bp2=bpl.find('sensor.camera.rgb')
bp2.set_attribute("image_size_x",f'{600}')
bp2.set_attribute("image_size_y",f'{600}')
bp2.set_attribute('fov','110')
#position of the sensor
ssp=carla.Transform(carla.Location(x=2.5,z=0.7))
sn=world.spawn_actor(bp2,ssp,attach_to=vl)
actor_list.append(sn)

In [ ]:

sys.path.append('C:/Users/ASUS/Downloads/CARLA_0.9.8/WindowsNoEditor/PythonAPI/carla')
import agents
from agents.navigation.global_route_planner import GlobalRoutePlanner
import carla
from agents.navigation.local_planner import RoadOption
# from agents.navigation.global_route_planner import GlobalRoutePlanner
from agents.navigation.global_route_planner_dao import GlobalRoutePlannerDAO

In [ ]:

#from agents.navigation.local_planner import RoadOption
#from agents.navigation.global_route_planner import GlobalRoutePlanner
#from agents.navigation.global_route_planner_dao import GlobalRoutePlannerDAO

# client = carla.Client('localhost', 2000)
# client.set_timeout(10.0)

world = client.get_world()
map = world.get_map()

dao = GlobalRoutePlannerDAO(map, sampling_resolution=1)
grp = GlobalRoutePlanner(dao)
grp.setup()

start_point = carla.Location(x=-77.887169, y=21.444204, z=1.805250)
end_points = [
    carla.Location(x=-6.446170, y=-79.055023, z=1.842997),
    carla.Location(x=65.516594, y=7.808423, z=1.843102),
    carla.Location(x=65.568863, y=4.308813, z=1.843102),
    # Add more locations as needed
]

longest_route = []
max_distance = 0

for end_point in end_points:
    try:
        print(f"Tracing route from {start_point} to {end_point}")
        route = grp.trace_route(start_point, end_point)

        if route is None:
            print(f"Route tracing failed for: {start_point} to {end_point}")
            continue

        if len(route) > max_distance:
            longest_route = route
            max_distance = len(route)
    except Exception as e:
        print(f"Error tracing route from {start_point} to {end_point}: {e}")

if longest_route:
    for waypoint in longest_route:
        world.debug.draw_string(waypoint[0].transform.location, 'O', draw_shadow=False, color=carla.Color(r=255, g=0, b=0), life_time=100.0, persistent_lines=True)
else:
    print("No valid route found.")


In [ ]:
# point=sp.location
# sampling_resolution=1
# smp=world.get_map()
# grp=GlobalRoutePlanner(world.get_map())
# dis=0

# for loc in spp:
#     print(loc.location)
#     cr=grp.trace_route(point,loc.location)
#     if len(cr)>dis:
#         route=cr
# for waypoint in route:
#     world.debug.draw_string(waypoint.transform.location,'O',draw_shadow=False,color=carla.Color(r=255,g=0,b=0),life_time=60.0,persistent_lines=True)



In [ ]:
# for waypoint in route:
#     vl.set_transform(waypoint[0].transform)
#     cv2.imshow('Fake self-driving',sn['image'])
#     cv2.waitKey(50)
    
time.sleep(2)
# cv2.destroyAllWindows()
sn.stop() # this is the opposite of camera.listen
for actor in world.get_actors().filter('vehicle'):
    actor.destroy()
for sensor in world.get_actors().filter('sensor'):
    sensor.destroy()